<a href="https://colab.research.google.com/github/igalindog/igalindog/blob/main/Connect/EjercicioNotas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# Instalar las librerías necesarias
!pip install --upgrade gspread google-auth

import pandas as pd
import gspread
from google.auth import default

# Autenticarte en Google (esto abrirá un cuadro de permisos)
creds, _ = default()
gc = gspread.authorize(creds)


In [51]:
# Abrir tu Google Sheet
spreadsheet = gc.open("Consulta_Notas_UnnecessaryDispatch")  # Reemplaza aquí

# Seleccionar la hoja (sheet) que quieres (puede ser .sheet1 o por nombre)
worksheet = spreadsheet.sheet1

# Leer los datos en un DataFrame
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# Mostrar las primeras filas
df.head()

,VehiculoPlaca,TipoServicio,Pais,Fecha,PONumber,Notes_Status,Notes_Timestamps,Notes_User,Notes_source
0,106344,towBreakdown,Puerto Rico,2025-03-20,4174716,"Nombre: Adriana Diaz \nTel: (787) 415-9665, (7...",2025-03-20 20:48:28.524000 UTC,Paloma Laureano,Notes
1,106344,towBreakdown,Puerto Rico,2025-03-20,4174716,"Se llama a Driver Kedvin (787) 216-7437, no co...",2025-03-20 21:17:11.533000 UTC,Marlin Nayibe Florez Forero,Notes
2,106344,towBreakdown,Puerto Rico,2025-03-20,4174716,"Se realiza seguimiento desde tracking, Driver ...",2025-03-20 21:29:39.083000 UTC,Marlin Nayibe Florez Forero,Notes
3,600960,towBreakdown,Puerto Rico,2025-03-06,4141230,Nombre: migelina \nTel: 7872386956 // 78750612...,2025-03-06 13:23:54.814000 UTC,Morelializ Dávila Valentín,Notes
4,600960,towBreakdown,Puerto Rico,2025-03-06,4141230,chofer indica que cliente no quiere grua que ...,2025-03-06 14:12:03.306000 UTC,Virmary Flores,Notes


In [52]:
# Supongamos que el DataFrame se llama df
# Contar los IDs de servicio únicos por cada placa
count_servicios = df.groupby('VehiculoPlaca')['PONumber'].nunique()

# Mostrar las placas con un solo ID de servicio
placas_con_un_id = count_servicios[count_servicios == 1].index

print(placas_con_un_id)



Index([  106344,   600960,   604247,   820377,   872905,   875321,   901474,
         918401,   940051,   942522,
       ...
       'hwo928', 'iDN251', 'jnj747', 'jum952', 'kcw474', 'kij436', 'kkd469',
       'kki216', 'klk646', 'knu043'],
      dtype='object', name='VehiculoPlaca', length=256)


In [53]:

# Eliminar del DataFrame las filas con placas que solo tienen un ID de servicio
df_filtrado = df[~df['VehiculoPlaca'].isin(placas_con_un_id)]

# Mostrar el DataFrame filtrado
df_filtrado.head()

,VehiculoPlaca,TipoServicio,Pais,Fecha,PONumber,Notes_Status,Notes_Timestamps,Notes_User,Notes_source
83,1179199,towBreakdown,Puerto Rico,2025-03-02,4132311,jaxmarie \n9392179370///7874772335\n\nemergenc...,2025-03-02 13:22:56.406000 UTC,Saydel Monet,Notes
84,1179199,towBreakdown,Puerto Rico,2025-03-02,4132311,PO Extracción 4132373,2025-03-02 14:10:04.320000 UTC,Laura Ximena Caballero Lamprea,Notes
85,1179199,towBreakdown,Puerto Rico,2025-03-02,4132311,Llama driver Armando Ortiz solicitando PO de e...,2025-03-02 14:13:35.879000 UTC,Laura Ximena Caballero Lamprea,Notes
86,1179199,towBreakdown,Puerto Rico,2025-03-03,4133824,Nombre: Jaxmarie Sanchez Colon \nTel: 93921793...,2025-03-03 10:34:57.492000 UTC,Morelializ Dávila Valentín,Notes
87,1179199,towBreakdown,Puerto Rico,2025-03-03,4133824,PO Extracción 4134265,2025-03-03 13:07:31.264000 UTC,Laura Ximena Caballero Lamprea,Notes


In [54]:
df.describe()

,VehiculoPlaca,TipoServicio,Pais,Fecha,PONumber,Notes_Status,Notes_Timestamps,Notes_User,Notes_source
count,1291,1291,1291,1291,1291,1291,1291,1291,1291
unique,324,1,1,32,407,1264,1291,52,2
top,KBK498,towBreakdown,Puerto Rico,2025-03-20,4196438,Se reasigna a petición de proveedor mediante e...,2025-03-22 15:24:09.041000 UTC,Mileyka De Jesús,Notes
freq,22,1291,1291,83,15,4,1,78,1290


In [55]:

# Agrupar por "Placa" y concatenar todas las "Notas"}
df_concatenado = df_filtrado.groupby("VehiculoPlaca")["Notes_Status"].apply(lambda x: " ".join(map(str, x.dropna()))).reset_index()

# Mostrar las primeras filas
df_concatenado.head()


,VehiculoPlaca,Notes_Status
0,1179199,jaxmarie \n9392179370///7874772335\n\nemergenc...
1,CDC061,Nombre: Jonathan Lopez\nTel: 7876742143 787627...
2,ETC144,(787) 388-3155 // veh se quedo corriendo medio...
3,EVX362,Noel Callejo 9394220299 \nTOW - en marcha vehi...
4,FKT149,Nombre: larazamy gonzalez \nTel: (787) 988-253...


In [56]:
import re

# Función de limpieza básica
def limpiar_texto(texto):
    texto = texto.lower()  # minúsculas
    texto = re.sub(r'[^a-zA-Záéíóúüñ\\s]', ' ', texto)  # mantener letras y espacios
    texto = re.sub(r'\\s+', ' ', texto)  # espacios múltiples a uno solo
    texto = texto.strip()  # quitar espacios iniciales/finales
    return texto

# Aplicarlo a tu DataFrame
df_concatenado['Notas_limpias'] = df_concatenado['Notes_Status'].apply(limpiar_texto)

# Ver resultado
df_concatenado.head()


,VehiculoPlaca,Notes_Status,Notas_limpias
0,1179199,jaxmarie \n9392179370///7874772335\n\nemergenc...,jaxmarie emergencia ...
1,CDC061,Nombre: Jonathan Lopez\nTel: 7876742143 787627...,nombre jonathan lopez tel ...
2,ETC144,(787) 388-3155 // veh se quedo corriendo medio...,veh se quedo corriendo medio de la carr u...
3,EVX362,Noel Callejo 9394220299 \nTOW - en marcha vehi...,noel callejo tow en marcha vehi ...
4,FKT149,Nombre: larazamy gonzalez \nTel: (787) 988-253...,nombre larazamy gonzalez tel ...


In [57]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

# Descargar stopwords en español si no las tienes
nltk.download('stopwords')
from nltk.corpus import stopwords

# Obtener las palabras de parada en español
stop_words_spanish = stopwords.words('spanish')

# Ver algunas de las palabras de parada
print(stop_words_spanish[:100])



['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
# Vectorización TF-IDF con las palabras de parada en español
vectorizer = TfidfVectorizer(stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df_concatenado['Notas_limpias'] )

# Mostrar las primeras características para revisar la vectorización
print(vectorizer.get_feature_names_out()[:10])


['abad' 'abierta' 'abierto' 'abre' 'ac' 'acabó' 'acacia' 'acc' 'acce'
 'acceso']


In [59]:
# Número de clusters (ajustable según lo que necesites)
n_clusters = 5  # Ajusta según el número de grupos que crees que pueden existir

# Realizamos el clustering con K-means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)

# Agregar la columna de clusters al DataFrame
df_concatenado['cluster'] = kmeans.labels_

# Mostrar las primeras filas del dataframe con los clusters asignados
df_concatenado.head()

# Mostrar las palabras más representativas de cada cluster
n_words = 10  # Cuántas palabras por cluster
for i in range(n_clusters):
    print(f"Cluster {i}:")
    # Obtener los índices de las palabras más relevantes para este cluster
    top_indices = np.argsort(kmeans.cluster_centers_[i])[-n_words:]
    top_words = np.array(vectorizer.get_feature_names_out())[top_indices]
    print(", ".join(top_words))
    print()


Cluster 0:
puerto, alto, rico, enciende, caguas, trujillo, recarga, des, pr, cayey

Cluster 1:
minutos, vehículo, eta, po, vh, driver, veh, chofer, servicio, cliente

Cluster 2:
ponce, evidencia, cliente, minutos, rio, grande, eta, driver, actual, servicio

Cluster 3:
puerto, nombre, dest, eta, juan, veh, inci, si, san, aplica

Cluster 4:
ori, ubi, enciende, po, san, vehi, eta, casa, pr, carolina



In [60]:
from sklearn.decomposition import LatentDirichletAllocation

# Realizar el modelado de tópicos con LDA
n_topics = 5  # Número de tópicos que quieres extraer
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# Visualizar las palabras más representativas para cada tópico
n_words = 10  # Número de palabras que mostrar por tópico
for index, topic in enumerate(lda.components_):
    print(f"Tópico #{index + 1}:")
    top_words_idx = topic.argsort()[-n_words:][::-1]
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
    print(", ".join(top_words))
    print()

Tópico #1:
pr, eta, cliente, si, ubi, casa, enciende, cayey, chofer, servicio

Tópico #2:
po, driver, veh, eta, extracción, rico, destino, puerto, servicio, trujillo

Tópico #3:
servicio, eta, cliente, san, carolina, veh, driver, juan, ubi, rico

Tópico #4:
servicio, eta, actual, si, grande, ubi, san, ori, rio, inci

Tópico #5:
vista, toa, alta, cqr, cx, buena, pago, responde, gulf, verde



In [37]:
!pip install nltk



In [61]:
import nltk
nltk.download('punkt_tab')

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer  # ESTA es la importación correcta
from sumy.summarizers.lsa import LsaSummarizer

def resumen_texto(texto, num_sentencias=3):
    parser = PlaintextParser.from_string(texto, Tokenizer('spanish'))
    summarizer = LsaSummarizer()
    resumen = summarizer(parser.document, num_sentencias)
    return " ".join([str(sentence) for sentence in resumen])

# Aplicar el resumen a todas las notas
df_concatenado['resumen'] = df_concatenado['Notas_limpias'].apply(resumen_texto)
print(df_concatenado[['VehiculoPlaca',  'resumen','Notas_limpias']].head())


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


  VehiculoPlaca                                            resumen  \
0       1179199  jaxmarie                           emergencia ...   
1        CDC061  nombre  jonathan lopez tel                    ...   
2        ETC144  veh se quedo corriendo medio de la carr      u...   
3        EVX362  noel callejo             tow   en marcha vehi ...   
4        FKT149  nombre  larazamy gonzalez  tel                ...   

                                       Notas_limpias  
0  jaxmarie                           emergencia ...  
1  nombre  jonathan lopez tel                    ...  
2  veh se quedo corriendo medio de la carr      u...  
3  noel callejo             tow   en marcha vehi ...  
4  nombre  larazamy gonzalez  tel                ...  


In [62]:
from sklearn.feature_extraction.text import CountVectorizer

# Usamos CountVectorizer para contar las palabras más frecuentes
vectorizer = CountVectorizer(stop_words=stop_words_spanish, ngram_range=(1, 2), max_features=10)  # Puedes ajustar max_features
X = vectorizer.fit_transform(df_concatenado['Notas_limpias'])

# Ver las palabras más comunes
words = vectorizer.get_feature_names_out()
word_count = X.sum(axis=0).A1  # Sumar las frecuencias de cada palabra

# Mostrar las palabras más frecuentes
freq_words = pd.DataFrame(zip(words, word_count), columns=['Palabra', 'Frecuencia'])
print(freq_words.sort_values(by='Frecuencia', ascending=False).head())


    Palabra  Frecuencia
3       eta         193
7  servicio         163
8       ubi         139
0   cliente         125
2    driver         120


In [63]:

# Configurar el vectorizador para extraer 3-gramas
vectorizer = CountVectorizer(ngram_range=(4, 4), stop_words=stop_words_spanish)

# Ajustar el vectorizador a las notas
X = vectorizer.fit_transform(df_concatenado['Notas_limpias'])

# Obtener los trigramas y sus frecuencias
trigramas = vectorizer.get_feature_names_out()
frecuencias = X.sum(axis=0).A1  # Aplanar la matriz

# Crear un DataFrame con los trigramas y su frecuencia
df_trigramas = pd.DataFrame({'Trigrama': trigramas, 'Frecuencia': frecuencias})

# Ordenar de más frecuentes a menos frecuentes
df_trigramas = df_trigramas.sort_values(by='Frecuencia', ascending=False)

# Mostrar los 20 trigramas más comunes
print(df_trigramas.head(20))


                              Trigrama  Frecuencia
4620               ori do remolque eta          21
5982              san juan puerto rico          19
2624              eta min pasajeros si          14
4266           min pasajeros si aplica          14
6081       servicio actual minutos eta          12
4285       minutos eta servicio actual          12
6620    tiempo servicio actual minutos          12
92         actual minutos eta servicio          12
84          actual eta servicio actual          11
6074      servicio actual eta servicio          11
4627          ori do remolque pasajero          11
6619        tiempo servicio actual eta          11
2185          do remolque pasajero eta          10
2656       eta servicio actual minutos          10
2170               do remolque eta min          10
4742               pas ori doremol eta           9
4747  pasajero eta coordinado contacto           9
5761              rico ori do remolque           9
4740                   pas eta 